In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})

In [4]:
customer = spark.createDataFrame(customer)
customer.show()

product = spark.createDataFrame(product)
product.show()

+-----------+-----------+
|customer_id|product_key|
+-----------+-----------+
|          1|          5|
|          2|          6|
|          3|          5|
|          3|          6|
|          1|          6|
+-----------+-----------+

+-----------+
|product_key|
+-----------+
|          5|
|          6|
+-----------+



In [5]:
from pyspark.sql.functions import count

customer \
.distinct() \
.groupby('customer_id') \
.agg(count('product_key').alias('products_count')) \
.where(f'products_count = {product.count()}') \
.select(['customer_id']) \
.show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+

